## Audio experiments

In [90]:
#>flags -d:blas=cblas -p:~/code/misc/jnim

# Original Python code:
# srate = 11000
# dur = 1000
# rr = np.arange(dur*2)
# wave = np.sin(2*np.pi*400*rr/srate) * np.sin(0.5*np.pi*rr/dur)
# plt.plot(rr, wave)
# Audio(wave, rate=srate, autoplay=True)
    
import arraymancer, pixie, sequtils, std/strutils, std/math, std/base64, std/strformat, wav, jupyternimpkg/display

proc pixieHTML(img: Image): string =
    return fmt"""<img src="data:image/png;base64, {img.encodeImage(PngFormat).encode}" />"""

proc plot1D(img: Image, arr: openArray[float]): Image =
    var path: seq[string] = @[]
    let r = float(img.height)/2.0
    for i in 0..<arr.len:
        path.add(fmt"{int(float(i)*float(img.width)/float(arr.len))} {int(arr[i]*r + r)} ")
    img.strokePath(fmt"M 0 {int(r)} L " & path.join(""), rgba(0, 0, 128, 255), strokeWidth = 1)
    return img

proc basicSine(srate: int): Tensor[system.float] =
    let srate = float(srate)
    let dur = 1000
    let rr_dur = dur * 2
    let rr = toSeq(0..<rr_dur).mapIt(it.float).toTensor()
    return sin(rr*(2.0*PI*440.0/srate)) *. sin(rr*(0.5*PI/float(dur)))

let srate = 11025
let wave = basicSine(srate).toSeq

#html loadWav("sample.wav").toHTML
html Audio(wave, srate, false)
html pixieHTML(newImage(400, 100).plot1D(wave))

<source src="data:audio/wav;base64,UklGRsQPAABXQVZFZm10IBAAAAABAAEAESsAAESsAAACABAAZGF0YaAPAAAAAAwAMABnAKkA7gAsAVkBbAFeASkBzgBPALP/A/9L/pr9//yK/Ef8Qfx//AL9xv3F/u//MgF7ArIDwQSRBRAGMAbpBTkFJQS8Ag8BOv9Y/Yn77fmi+ML3YPeK90L4g/k++1r9uf80AqUE4gbDCCUK7goKC3EKJwk8B8kE8gHl/tD75vha9lj0BfN88szy9/Pu9Zn4z/tg/xMDrQbzCa0MqQ7ED+UPAw8oDWsK9Ab4ArT+a/pl9uPyI/BW7p3tC+6f70Xy2PUj+uP+zQOTCOYMfRAaE48UvRScEzgRsg09CR4Ep/4r+QX0i+8H7LfpxuhI6T3riu7/8lj4RP5jBFUKtw8xFHYXTBmRGTkYVRUNEaILZgW9/g/4yfFR7AToK+X544jk1Oa+6g3wcPaD/dUE8gtmEscXuRv5HV0e2Bx9GXwUIQ7OBvf+GPew7znpHuS04Drfy99l4uXmBu1r9KD8IgVqDfIUPRvjH5QiHyN2Ia0d+xe4EFQIVP9G9r3tROZV4FbcitoV2/XdAOPq6UrynPtKBb0OWReSHvEjGifVJxAm4yGKG2YT+AnU/5r17+ty463cEdjt1WnWhNkS373mEPB4+k8F6Q+aGcQh4CeIK3sspCocJiUfKha4C3QAE/VJ6sbgJtno02XRytEX1R3bgOO+7Tb5MAXvELQb0SSvK9wvDzEvL1UqyiIBGZMNNwGx9MnoQN7D1d7P9Mw5za/QJNc14FXr1ffuBM8RqB24J1wvEzSONa4zjS52JugbiA8ZAnT0cOfi24XS9MucyLnIT8wp09/c2OhZ9okEiBJzH3gq5TItOPY5HzjAMikq3x6VERwDXPQ/5qzZbs8syGHETsT6xy/Pf9lI5sH0AgQaExUhEC1INiY8RT6APOw23i3kIbgTPARn9Dfln9d+zInERMD5v7LDOMsY1qfjEPNbA4cTjiJ+L4Q5/D95Qs1ADzuVMfMk7xV5BZf0VuS91bjJDMFIvL27er9Hx63S9+BG8ZMCzRPeI8IxmDyuQ45GBEUlP0k1Cig5GNEG6fSd4wTUHMe3vW64nbdVu1/DP8873mbvrQHvEwUl2zOBPzlHgkojSSxD+TgpK5UaRAhd9Qvjd9KsxIu6ubSas0S3gb/Ry3Tbce2pAOsTAibINT9CnEpUTidNIkeiPEsu/hzQCfL1oeIU0WjCibcrsbivS7Oxu2bIpNhp64r/xBPVJok30UTWTQJSDlEFS0JAbzF1H3ILp/Zd4t3PUcC0tMWt96tsr/C3AMXO1VDpT/56E38nHTk2R+RQiVXVVNBO1kOTNPchKg1690Di0c5nvgyyiapbqKmrQrSiwfTSKOf6/A0TASiEOmxJxVPnWHtYhFJcR7M3gST2Dmv4SOLwzay8k695p+WkBaipsE2+GdDy5I37fxJaKL47c0t4Vhtc/VscVtFKzjoRJ9MQePl14jvNILtJrZakmKGBpCitBbs+zbLiCvrREYwoyzxKTfxYJF9ZX5ZZM07iPaYpwBKg+sXir8zCuS+r4qF0niChwKnMt2bKaOBz+AQRliirPfFOUFv/YY5i8Vx/UetAPCy7FOH7OONOzJO4Rqldn3yb5Z10pqO0lMcY3sj2GhB6KF8+aFBzXatkmWUrYLNU6EPSLsIWOf3N4xbMk7eOpwqdspjRmkajj7HJxMLbDvUVDzko5j6uUWNfJmd4aEBjzVfWRmYx0xim/oHkB8zCtgim6JoXluWXOaCQrgjCa9lE8/UN1CdBP8RSIWFxaSprL2bLWrJJ9DPrGicAVeUgzB+2taT5mKyTJJVOnaqrVL8T123xvAxLJ3E/qFOsYolrrm32aKlde0x7NgkdugFF5mDMq7WUoz6Xc5GQkoia3qiuvL3UjO9sC6Emdz9dVANkbW0BcJNrZ2AuT/k4Kh9eA1LnxcxktaWit5VsjyqQ6Zcvphq6bNKi7QcK1yVTP+FUJ2UdbyNyBW4CY8hRajtMIQ8FeOhQzUq16aFllJmN9I1ylZ+jmbch0LHrjwjuJAY/NVUWZplwEnRJcHhlSVTOPW0jzQa36f3NXLVfoUiT+4vuiyaTMKEttd/NvOkFB+cjkj5aVdJm3nHOdV9yxmesViBAiyWUCA3rzc6ZtQahYJKSihuKBZHkntqyqcvF52wFxSL4PVFVWWfuclV3RHTsafFYYEKjJ2MKeOy9zwC236CtkV+Je4gTj72coLCAyc7lxgOIITk9GlWtZ8hzpnj4dehrFVuLRLMpOAz17czQkLbooC+RYogQh0+NvJqCrmbH2eMUAjQgVjy2VM5na3TBeXp3uG0WXZ9GuCsRDoPv+NFJtyKh55Cdh9qFu4vkmIOsXsXp4VkAyR5ROydUvWfYdKV6x3hab/NemkiyLeoPIPE/0ye4iqHTkA+H2oRaijeXpKprw//fmP5KHSw6bVN6Zw91U3vgec1wqmB5Sp0vwxHK8qHUK7kgovOQt4YRhCuJtZXnqI3BH97R/Lkb6DiKUgVnD3XIe8N6D3I4YjtMdzGYE370GtZSuuSiR5GXhn+DMIhglE6nyL9J3Aj7FxqHN4BRYGbbdAZ8cHshc51j3U0+M2gVO/ap15u706POka6GJINqhzuT2qUdvoHaPvlnGAs2T1CNZXF0DXzme/9z12RfT/A0MRf/90vZBL3spIeS+4YBg9mGRJKNpI+8yNh296sWdjT5Totk0nPbeyV8qnTkZb5QizbvGMb5ANuLvi+mcZN/hxaDfoZ/kWqjHrsh17L15RTKMoBNXGMBc3N7LHwhdcNm+FENOKIaj/vE3C7AmKeLlDiIY4NahuyQcKLOuY7V8/MXEwgx5ksCYvxx1Hr8e2N1c2cMU3Q5RxxY/Zbe7cEoqdSVJonng2yGjJCjoZ+4ENQ+8kQRMy8sSn9gxnD+eZR7b3XzZ/hTvzrcHR//c+DDw9yqSpdHiqOEtoZfkAKhk7eq0pLwbQ9OLVVI015fb/N49HpFdUJou1TqO14f4ABZ4rDFsqzsmJyLlYU3h2eQj6Cttl3R8+6VDVkrYkYAXcpts3ceeuZ0X2hUVfY8zCCbAkbksseorrmaI42+hu+Ho5BLoOy1K9Bj7b4LWClWRAlbBmw/dhB5T3RJaMFV3z0lIk0EOObFybywrpzajhyI34gUkTegVLUXz+Tr6glMJzJC71gWaph0zHeCcwFoAVamPmUj9AUs6OnL7bLKnsCQr4kEirqRU6DktCLOd+ocCDkl+j+0Vvxnv3JSdn9yhWcUVkc/iySOByHqGs45tQyh1ZJ1i2CLlpKhoJ+0TM0e6VUGHyOuPVlUuGW2cKR0RnHVZvlVwj+WJRkJFOxW0Jy3caMVlW2N8oynkyGhhLSZzN3nmAQCIVE74lFOY31uwXLXb/BlrlUWQIMmkgoC7pzSFrr3pYCXl4+4ju6U06GWtAnMs+bnAuMe5jhRT75gGGyscDNu2GQzVUFAUif4C+vv6NSivJyoFJrxkbKQaZa3otW0nsuk5UQBxRxvNqZMCl6GaWVuW2yMY4hUQkABKEoNy/E610G/X6vPnHmU4JIZmM6jQbVYy7Dksf+qGu0z5kk2W8tm7WtPagxirVMaQI4ohA6g843Z7sE8rq+fLpc/lfyZF6XbtTnL2eMv/pUYZDERR0JY52NHaRBoWWChUsY/+CimD2j14dupxDKxsqIOms6XEpyTpqS2Qssh48H8hhbWLitEMlXcYHNmoGVzXmRRRj89Ka4QIvcy3m3HPrTVpRidjZpbnkConLd0y4riafuCFEUsNUEHUq5dc2P/Yltc9k+ZPl4pmhHM+H/gOspetxipSaB6ndWgIKrCuM/LE+In+ogSsykzPsNOXVpKYC9gEVpYTsA9WClpEmP6xuIMzZC6dqyfo5Ogf6MwrBi6Vcy/4f/4nRAiJyY7akvsVvhcMV2XV4lMujwsK

<img src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAZAAAABkCAYAAACoy2Z3AAAsdUlEQVR4Ae2dC7QlVXnnf/tcGhoJIkg3ow4IKMKIirxfwqkW5NWKqISgExZkkcAE4wOlFZdC060oaV+EqCyIQ2RkkIdhGGARQpIhdeX9cmjHVsJCAeUVlbZBHk3Tt2b99646Vafud27X6fvoe+7d/7XOgvvvqjpVu3Z9v/N9e1fVEFFRA6fFG0E60r3bZ58F7Takw6U/3b2oqMFWBEjUNNLpm8Kdr3TvkAWB9t9D++3dsGifDDwzgd4JwKra9iyvHfa3Cq6mnnVslhcVNT0VARI1BWoKhsOvgvZbuoN28kngqZp3PvDjyfXa5wA/7/4O07OgYnjJccDzte01hI8FFcuLippaRYBEjUMWGD67Bdy2unujTcCwaHOYcxFwaxlkLW/xjuDOhuyWcrkJ9zYGBKnhyncYnmRCxYLPIuCX5fYkCzRNoWItZ0HF8qKiJkYRIFENZcHCAsMht0N7mzJQWhCwvEOOhNYJ3WCwvGQhuA92Lzfh3m7QOq32HYZnQcXypPYy4O6yXSQTNBOcvTQFTVRU/4oAiTLUJIuwILD4XdD6fHfgtSBgeSrxuAXd61pe+/2MWm6iPQ+ko2v7InDVPQMqlve5eTDnnO7t9QRNw+xFbbO+JbEIlaiJUQTIrJcFiyZZhAWB9omjg7sCXSMwtEevu8E8waIGM8szoaJ1a6BZsDe0TuoGiAWanlCxshejJKa2jlCJmjpFgMwqNYFF0yxCwWpUkJ1OEJiKfREsGoBmgZZbWAOIvBpoLKhY2YvUFCpWSUxtI03UzLGo2aoIkBmr9YVF0yxCQWhU8JyKoD2I33GEkV0JIDX4WFCxspe+oGKVxIxxFrWrFKES1VwRIDNCEwkLBZJ6YIteuElx0tvFgIqVvTSFSs+SmAEVZZSx/BXVnyJABlKnzIH7Kndi18csrDKUlVlEMEwRGMaTNRnZS1OoWCWxnlCJYypRfSsCZNqrDgvB4Y8+BOkdYdf1d33mU4TFAIBBNxG+A5xuVrw5nMvG3hGQvR3cv0P6T2FdQSU7BNz/Kz2rJGZBRWo6phIzlahSESDTSuuExXaQ3QncVd5PEGExdbBgD3AroO+A3wsW2wJrIP1h6IZ+uabeNoCD9Kp83SOALWveQuBt4B6G9KawnKDC4flx5F4xpsLycjlpQ0ElalAUATJtpEyiK7OwYPFxaB3WPZtHQWfSa/PjKcEMwLomGPbIg2w1O1BwfwHSa0K3aQsM8tYTAm5zcPMhvaTcXmNvLrgdIdXd+wVELW9LyDaD9HItRZiO/KpuT2Mq7sOQ6dlf+XJjQaUKJGmiy1/x7vlBUQTIBlM124iw2MBZhCBggaHubQluC0gvDd1GAGkc8A1YkAF7QarHpuRt0Nh7Edy7If1rrRnaz/Kc/nk3SL+l/wnL1T1f/tKjWt5ULjcWVDI9tDIHjaRMxT0K6Y36K0hQcc/WMhoBlzJ7lpp6ESrTUREgU6IqLKR6thEzizyw1aa1Krg09cyMYY+GWYQBBstzcyDbBdJv6yyGfW4c8A1YuBfw5aT558GKLGyvqcdqcPtDciGkq8O6pvcccAQky8Ij8NsK4nVP5a8nu5frBRXeDby6BE2RqWSCbQ5WSVDR/nSB5gRQltQFGu1PhMqAKgJk0lWHhZltDECZp2kgn4rlGoPByiwMWFhgsDwFRB1fel7oNv68jQcCgsrOMP8KSJ/LIdDU03DH62BtCsNP5+tanmbrbQ/uRkifyZdr4vWAihN8DiqB5DOVPYEdIP2a9gAPlaEvaGdK0Eh++nAlg5M0KO/+qJa9jAMqUVOpCJBJlQmLOI4RAm/DbKMxLAwwWF5TMFiw0FiHOwCSCyBdE46jccC3wKAYOw9G7oThx/Pt9eExF1qPQPpgua7paczjIRh+uLJcE68OGkFFAKkASZmKSn3sCgdcCre+gIdKaydglxI0mj7svg4eIDloJJW60DhQJXsRVHgtpNdriaCmUImlrqlUBMiE6ayFkJwBB2cw/FDYbIRFCFhTAQsLDJbXGAxrGA0LaQdYez38aGU4NmlcENgUWr+GdEW5vaaeyx+bn95Vrmt6LwItSO+uLNfQWydoBJXnwW0WyoXDjxKgIvjMKzMfTR/O9g/lrwI0ki91zSuzF8lDZXtIL9ZfQW1lJdtFqEwvRYBMmJLjoXVG90uI1OmbBs/Zttx4MovGYNA4wQK6Sk79gMGExdbQuhfSx8pg3DTgW8sp4GcZoyDQxOMlfMahx4+or3lZnmAxNwxed5Zr6DUFjZZzKlndAwVUqvDx04eHwL1SgkbjJ0On+b0eBRX3Bki+Cuna8O8aP1GfmSioRE2EIkDWS2a2EccxfHBSwK6+7EntYt4wN47xicZgWAPuAEgugHQNtNsE7QBrr4cfrSw9tzW07oX0sbE95oJ7AtKfVpZbDVkG6V39e6wGNoF0uFy3sZera91cE+a9iD/mru1Z3vPAq7qPw2m5FqR3Q3shsArvOQfpPbBgb2i9FdxvwK2A4Ufhc/Ng6DTgF+DugvRJ/BG1zwHeBskySNd6i+Q4YB9ILw5/S/4cvxnSa/VXkPcI+1bo7LNCe+u4otZXQ+u74uzRWQshOQMOzmD4oXDcyfHQOgP4edkp1UndAshumd0e7wCndrk5tJXahW2BNZD+sOJtCW4LSC8tPTcHsl0g/XbpsRrfrul5FW8NuAMguQDSdYChKSwsMFieWwOZguIdIQB5TXcI5FKw9Oejokn1BJC54XrofG8FNAsEFUkAqUGFFeBGIL0fFm8MLAMeBnc7JVQWAXt2Q0Xf4z4A6fnacJC8JlCJ6ldD/a4w+xRhEQKlBcemsHCbg5sP6SWh/+iCngxYjCuLMMBgeR4Wc2pBMdekBuNcM+o7jgAPFAsqTwCbh3ZOdgMOBO4poSK1l4FbCehHW5GpnACtd0P7+5Dq34r+2wAqMSvpVxEgXbKyDR/sYmYxLliQAXuVvwrVpgrEatf1zSwsWOjXbr28ZHmNwaDMIsLCa8rAVUAll743UaaiWPUYHah0Sl3/WIPKOZD9DtxjkC4PG+kLKrHU1Yd0UqI6itlGyDbGk1lYsHAvAIfD/PNgRZZ/Rx9lqKawsMBgeREMpRSgPdArmnaeALKq3EHtXzF+wgMlVDqlru8Brw7Zi2RCpQ2tDxiZyu6QXqm1grScpO+MqmsWAyRmG3kgbzhmYZWhGsNCGcjOMP8KSJ/Lv1dqOJgds4hSCmRFUCs0G73O+InKX3nW0Cl1/VROAEhPqJwA6pMCUCdTaYP7M9jnArj95bAN79WgEktdhWYJQCxYzMZsY0PCws2DkTth+PESIOPJLGIWUWo2AmTMUleerahdLKhIykr4MfBSAI0kqLh3wND9MPygnNBXLahI2v7s1iwBSIRFuBCs2VCTkVlYsGBTaP0a0hX5vkjWlFhjMDvCopSCVruYjpwregECVqmrM35SgUonK7kOWBvWlTxU7s/HVIblhL7aOgyG7qpBJWYlMBMBYmUbuuA0YDtTp9hOt8zCgoV5r0SERUcKWOqnVUUvBPdxtYsxfmJlJT2hcjJkm4J7OuyLpP2xspI6VGa+ZmAGMtOzjUGFhc8irBvm4iwnrwiLEKAVnKuajHaxshILKlKi+0l+BKwM+ydpH82spDZ+MvPHSgYcIDM925hRsMg1GQFBF3RV8TtCsIvtEkpQVfm+YWUl8hQPq6WuHYETgf8TtqB1pV5ZiQUVjum+U35mQUUNNsCaSdlGhEV3R9TFGgNgjwAY22ri+4YAksNDUv8TVDLFyGfkBChLZlbSo9TV+gi0/zU8Ikfy564GlcGVGmdANOOzDeNxHxtygNsaszDLULl0wfmLo6LohaAT22USAv5UQPT9UMBDUn/ulZWYUNG0YN2E/EjoB5K8OlQGNysZIIDMqGzjbeAehvQmdalwcUVYhLYopItVF1t